In [1]:
import cnvpytor
import subprocess

print(cnvpytor.__version__)

1.3.1


In [2]:
filename = "test_with_del__3_formated.fa"
command = f"cat {filename} | wc -l"
result = subprocess.check_output(command, shell=True)
line_count = int(result.strip())
line_count

165707

In [3]:
read_count = line_count * 50 * 30 / 202
read_count = round(read_count)
read_count

1230498

In [4]:
command = f"wgsim -d 300 -N {read_count} -1 101 -2 101 {filename} frag1Copy_test.fastq frag2Copy_test.fastq"
result = subprocess.check_output(command, shell=True)
print(result)

[wgsim] seed = 1706517540
[wgsim_core] calculating the total length of the reference sequence...
[wgsim_core] 1 sequences, total length: 8285284


b'chr22\t878\tG\tC\t-\nchr22\t1600\tG\tR\t+\nchr22\t2299\tT\tA\t-\nchr22\t4781\tG\tT\t-\nchr22\t4855\tA\tR\t+\nchr22\t6192\tG\tS\t+\nchr22\t8425\tC\tS\t+\nchr22\t8602\tT\tG\t-\nchr22\t9738\tT\tW\t+\nchr22\t11330\tC\tS\t+\nchr22\t12184\tT\tC\t-\nchr22\t14457\tG\tC\t-\nchr22\t14505\tG\tC\t-\nchr22\t15393\tT\tY\t+\nchr22\t16534\tT\t-\t-\nchr22\t16948\tG\tT\t-\nchr22\t22980\tA\tW\t+\nchr22\t23058\tA\tG\t-\nchr22\t24423\tT\tG\t-\nchr22\t24652\tG\tR\t+\nchr22\t27567\tT\tW\t+\nchr22\t27959\tG\tC\t-\nchr22\t29226\tA\tG\t-\nchr22\t30812\tG\tT\t-\nchr22\t31408\tG\tR\t+\nchr22\t32195\tG\tT\t-\nchr22\t33415\tT\tK\t+\nchr22\t34820\tA\tT\t-\nchr22\t38243\tGG\t-\t-\nchr22\t38506\tG\tA\t-\nchr22\t39569\tAG\t-\t-\nchr22\t39760\tA\tC\t-\nchr22\t40315\tA\tM\t+\nchr22\t40962\tC\tG\t-\nchr22\t41251\tG\t-\t-\nchr22\t41664\tG\tA\t-\nchr22\t43473\tC\tT\t-\nchr22\t43855\tT\tY\t+\nchr22\t44790\tG\tK\t+\nchr22\t45570\tT\tW\t+\nchr22\t46014\tG\tK\t+\nchr22\t46173\tT\tK\t+\nchr22\t46307\t-\tA\t+\nchr22\t47763\t-\t

In [5]:
# bowtie related stuff
!bowtie2-build reference.fa bias
!bowtie2 -p 4 -k 15 --no-mixed -x bias -1 frag1Copy_test.fastq -2 frag2Copy_test.fastq -S biasOut.sam

Settings:
  Output files: "bias.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  reference.fa
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:01
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:01
bmax according to bmaxDivN setting: 12826141
Using parameters --bmax 9619606 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 9619606 --dcv 1024
Constructing suffix-array element

In [6]:
# sam to bam file conversion
!samtools sort biasOut.sam -o biasOut_sorted.bam
!samtools index biasOut_sorted.bam

[bam_sort_core] merging from 1 files and 1 in-memory blocks...


In [7]:
app = cnvpytor.Root("test.pytor", create=True, max_cores=8)

In [8]:
app.rd(["biasOut_sorted.bam"])

In [9]:
app.calculate_histograms([100, 1000, 10000, 100000, 1000000, 10000000])

In [10]:
app.partition([100, 1000, 10000, 100000, 1000000, 10000000])

In [11]:
calls = app.call([100, 1000, 10000, 100000, 1000000, 10000000], print_calls=True)

duplication	chr22:643201-644800	1600	1.4898	6.708786e-06	1.220945e-57	1.000000e+00	1.000000e+00	0.0000	1.0000	0
duplication	chr22:1306101-1325300	19200	1.3990	0.000000e+00	4.358285e-195	0.000000e+00	6.366031e-242	0.0000	1.0000	0
duplication	chr22:2926201-2963000	36800	3.1413	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:4546401-4713300	166900	1.6705	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:6655001-6661800	6800	2.4355	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:6848301-6854500	6200	1.6812	0.000000e+00	1.018620e-238	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:8039601-8047700	8100	1.9714	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:9424201-9461600	37400	1.4917	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:13230601-13395900	165300	1.4113	0.000000e+00	0.000000e+00	0.0000

In [12]:
for bin_size in calls:
    print("Bin size: ", bin_size)

Bin size:  100
Bin size:  1000
Bin size:  10000
Bin size:  100000
Bin size:  1000000
Bin size:  10000000


In [13]:
print(f"{filename=}, Histogram Size: 100")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[100]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__3_formated.fa', Histogram Size: 100
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,643201,644800,1600,1.4897881902780294,0.0
duplication,chr22,1306101,1325300,19200,1.3990201675436267,0.0
duplication,chr22,2926201,2963000,36800,3.1413013986390834,0.0
duplication,chr22,4546401,4713300,166900,1.670529258135872,0.0
duplication,chr22,6655001,6661800,6800,2.435499823175638,0.0
duplication,chr22,6848301,6854500,6200,1.6811503404768295,0.0
duplication,chr22,8039601,8047700,8100,1.9713934305189968,0.0
duplication,chr22,9424201,9461600,37400,1.4916797831757218,0.0
duplication,chr22,13230601,13395900,165300,1.4112850129224155,0.0
duplication,chr22,13449001,13453700,4700,1.3276743605088468,0.0
duplication,chr22,13480801,13512800,32000,4.241101282506889,0.0
duplication,chr22,15242601,15246400,3800,3.0262068809801845,0.0
duplication,chr22,18664801,18665900,1100,1.4941028968180294,0.0
duplication,chr22,18680801,18682700,1900,1.4391597488570296,0.0
duplica

In [14]:
print(f"{filename=}, Histogram Size: 1000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[1000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__3_formated.fa', Histogram Size: 1000
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,1306001,1325000,19000,1.4046780057929107,0.0
duplication,chr22,2926001,2963000,37000,3.1398717770754927,0.0
duplication,chr22,4546001,4713000,167000,1.674632995959561,0.0
duplication,chr22,6655001,6662000,7000,2.406986958434764,0.0
duplication,chr22,8040001,8048000,8000,1.9590274573089375,0.0
duplication,chr22,9424001,9462000,38000,1.490713190199794,0.0
duplication,chr22,13231001,13396000,165000,1.4152230985499417,0.0
duplication,chr22,13481001,13513000,32000,4.249655344606035,0.0
duplication,chr22,18705001,18759000,54000,2.6448615528087647,0.00018207232443770945
duplication,chr22,18760001,18881000,121000,2.7908675587256413,0.00012478202633374612
duplication,chr22,19259001,19268000,9000,3.2536447696731523,0.0
duplication,chr22,21248001,21381000,133000,1.4929297102590295,0.00012744417347807096
duplication,chr22,21533001,21548000,15000,1.303461963230975,0.00266

In [15]:
print(f"{filename=}, Histogram Size: 10,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[10000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__3_formated.fa', Histogram Size: 10,000
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,4550001,4710000,160000,1.6669663958557959,0.0
duplication,chr22,6610001,7010000,400000,1.0954663623503027,0.0
duplication,chr22,7960001,8040000,80000,1.068591384052808,0.0
duplication,chr22,13230001,13410000,180000,1.3796936797128616,0.0
duplication,chr22,18710001,18880000,170000,2.7439706373148964,0.00014548387133727172
duplication,chr22,21250001,21380000,130000,1.474140809883482,0.00012751364667325035
duplication,chr22,21490001,21640000,150000,1.3190468732520928,0.0004201957064859154
duplication,chr22,25150001,25310000,160000,1.2762069806553893,0.00012919104223242122
duplication,chr22,28470001,28600000,130000,3.062402840712097,4.27468874060775e-05
duplication,chr22,30800001,30980000,180000,2.8861412185576545,2.598475993829619e-05
duplication,chr22,31150001,31360000,210000,1.4201439351140415,4.966088708533159e-05
duplication,chr22,31870001,32250000,380000,

In [16]:
print(f"{filename=}, Histogram Size: 100,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[100000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__3_formated.fa', Histogram Size: 100,000
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,3000001,6800000,3800000,1.0757016213081938,0.0
duplication,chr22,45600001,47000000,1400000,1.4424141307036282,6.173754616859513e-05
duplication,chr22,47100001,50100000,3000000,1.6988266844365862,5.174451711400623e-05


In [17]:
print(f"{filename=}, Histogram Size: 1,000,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[1000000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__3_formated.fa', Histogram Size: 1,000,000
type,chromosome_name,start,end,size,cnv_level,q0


In [18]:
print(f"{filename=}, Histogram Size: 10,000,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[10000000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__3_formated.fa', Histogram Size: 10,000,000
type,chromosome_name,start,end,size,cnv_level,q0


In [19]:
del app